In [2]:
import torch.nn as nn

In [3]:
import os, random, torch
import numpy as np

def setSeeds(seed = 42):
    # 랜덤 시드를 설정하여 매 코드를 실행할 때마다 동일한 결과를 얻게 합니다.
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)    
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.cuda.manual_seed_all(seed) # if use multi-GPU
    torch.backends.cudnn.benchmark = False

In [4]:
setSeeds()

# 원래 Inference (모델 로드마다 시드 초기화 안할 때)

In [8]:
# Model 1
cate_embedding = nn.Embedding(4, 10)
cate_embedding.weight

Parameter containing:
tensor([[-1.4570, -0.1023, -0.5992,  0.4771,  0.7262,  0.0912, -0.3891,  0.5279,
         -0.0127,  0.2408],
        [ 0.1325,  0.7642,  1.0950,  0.3399,  0.7200,  0.4114,  1.9312,  1.0119,
         -1.4364, -1.1299],
        [-0.1360,  1.6354,  0.6547,  0.5760,  1.0414, -0.3997, -2.2933,  0.4976,
         -0.4257, -1.3371],
        [-0.1933,  0.6526, -0.3063, -0.3302, -0.9808,  0.1947, -1.6535,  0.6814,
          1.4611, -0.3098]], requires_grad=True)

In [9]:
# Model 2
cate_embedding = nn.Embedding(4, 10)
cate_embedding.weight

Parameter containing:
tensor([[ 0.9633, -0.3095,  0.5712,  1.1179, -1.2956,  0.0503, -0.5855, -0.3900,
          0.9812, -0.6401],
        [-0.4908,  0.2080, -1.1586, -0.9637, -0.3750,  0.8033,  0.7165,  1.5335,
         -1.4510, -0.7861],
        [-0.9563, -1.2476, -0.7499, -0.5922, -0.8146, -1.0212, -0.4949, -0.5923,
          0.1543,  0.4408],
        [-0.1483, -2.3184, -0.3980,  1.0805, -1.7809,  1.5080,  0.3094, -0.5003,
          1.0350,  1.6896]], requires_grad=True)

똑같은 초기값 임베딩을 기대했지만, 그렇지 않습니다

k-fold에서 생성된 1번 모델에서 임베딩 레이어와 2번 모델에서 임베딩 레이어의 값이 다르기 때문에,

2번 임베딩 레이어에서 학습시킨 값과 다른 값이 입력으로 들어갑니다

# 수정한 Inference (모델 로드마다 시드 초기화)

In [10]:
# Model 1
setSeeds()
cate_embedding = nn.Embedding(4, 10)
cate_embedding.weight

Parameter containing:
tensor([[ 1.9269,  1.4873,  0.9007, -2.1055,  0.6784, -1.2345, -0.0431, -1.6047,
         -0.7521,  1.6487],
        [-0.3925, -1.4036, -0.7279, -0.5594, -0.7688,  0.7624,  1.6423, -0.1596,
         -0.4974,  0.4396],
        [-0.7581,  1.0783,  0.8008,  1.6806,  0.0349,  0.3211,  1.5736, -0.8455,
          1.3123,  0.6872],
        [-1.0892, -0.3553, -1.4181,  0.8963,  0.0499,  2.2667,  1.1790, -0.4345,
         -1.3864, -1.2862]], requires_grad=True)

In [11]:
# Model 2
setSeeds()
cate_embedding = nn.Embedding(4, 10)
cate_embedding.weight

Parameter containing:
tensor([[ 1.9269,  1.4873,  0.9007, -2.1055,  0.6784, -1.2345, -0.0431, -1.6047,
         -0.7521,  1.6487],
        [-0.3925, -1.4036, -0.7279, -0.5594, -0.7688,  0.7624,  1.6423, -0.1596,
         -0.4974,  0.4396],
        [-0.7581,  1.0783,  0.8008,  1.6806,  0.0349,  0.3211,  1.5736, -0.8455,
          1.3123,  0.6872],
        [-1.0892, -0.3553, -1.4181,  0.8963,  0.0499,  2.2667,  1.1790, -0.4345,
         -1.3864, -1.2862]], requires_grad=True)

모델 1과 모델 2에서 임베딩이 똑같이 되는 것을 알 수 있습니다

그렇기 때문에 k-fold를 inference할 때, 각 모델을 로드할 때마다 시드를 초기화해야 임베딩 레이어의 초기값을 의도했던 대로 가져올 수 있습니다